# Migrating to Pydantic

It makes sense to migrate to a well supported package for handling metadata, like `Pydantic`.  

This will also remove the costly step of reading the JSON files each time an object is initiated and weird things that happen when you set `_attr_dict`.  

Steps:

1. Convert existing JSON to JSON Schema files and move to `mt_metadata.mt_metadata.standards` folder for reference later.
2. Convert the JSON Schema files to `pydantic.BaseModel` objects using `datamodel-code-generator`

## 1. Convert existing JSON

This is relatively straight forward just a simple mapping and inclusion of some other things.  The mapping is being done in `mt_metadata.utils.converters.to_json_schema`.

## 2. Convert JSON Schema to pydantic.BaseModel

This is tougher.  For now use `datamodel-code-generator`.

```
datamodel-codegen.exe --input .\person_schema.json --input-file-type jsonschema --field-constraints --formatters black --use-annotated --output person_model.py
```

In [ ]:
from typing import Literal, Optional, Annotated
from pydantic import BaseModel, Field, ConfigDict
import json

In [2]:
def custom_docstring():
    def decorator(cls):
        cls.__doc__ = cls.model_json_schema()["properties"]
        return cls
    return decorator

# @custom_docstring()
# class Order(BaseModel):
#     order_id: int
#     amount: float

# print(Order.__doc__)

In [ ]:
class DocPreservingModelMetaclass(ModelMetaclass):
     def __new__(mcs, name, bases, namespace, **kwargs):
         doc = namespace.pop('__doc__', None)
         cls = super().__new__(mcs, name, bases, namespace, **kwargs)
         cls.__doc__ = doc
         return cls

In [ ]:
class Dipole(BaseModel):
    model_config = ConfigDict(validate_assignment=True, use_attribute_docstrings=True)

    length: float = Field(
        default=0, 
        description="dipole length", 
        examples=10,
        required=True,
        json_schema_extra={"units": "m", "required": True},
    )
    azimuth: float = Field(
        default=0,
        description="measurement azimuth clockwise positive from north",
        examples=36,
        json_schema_extra={"units": "degrees"},
    )



class EX(BaseModel):
    """
    EX
    """

    model_config = ConfigDict(validate_assignment=True, use_attribute_docstrings=True)

    type: Optional[str] = Field(
        default="", 
        description="name of person", 
        examples="ann",
        required=True,
    )
    name: str = Field(
        default="EX",
        description="The name of the EX.",
        title="Name",
    )
    description: str = Field(
        default="a",
        description="A brief description of the EX.",
        title="Description",
    )

    dipole: Dipole = Dipole()

    #__doc__ = json.dumps(EX().model_dump_json(), indent=4)

    @property
    def other(self):
        return 10

    def __pydantic_init_subclass__(cls) -> None:
        cls.__doc__ = cls.model_json_schema()

    def __str__(self):
        return json.dumps(self.model_dump_json(), indent=4)
    
    def __eq__(self, other):
        if isinstance(other, EX):
            return self.model_dump() == other.model_dump()
        return False

In [49]:
a = EX(name="a")
b = EX(name="a", description="new")


In [50]:

c = a.model_copy()


In [55]:
sorted(c.model_fields.keys())

['description', 'dipole', 'name', 'type_']

In [66]:
f = c.dipole.model_fields["length"]


In [70]:
f.is_required()

False

In [26]:
with open(r"c:\Users\peaco\OneDrive\Documents\GitHub\mt_metadata\examples\notebooks\person_schema.json", "w") as fid:
    json.dump(a.model_json_schema(), fid, indent=4)

In [25]:
with open(r"c:\Users\peaco\OneDrive\Documents\GitHub\mt_metadata\mt_metadata\timeseries\standards\person.json", "r") as fid:
    d = json.load(fid)

In [ ]:

def convert_to_json_schema(old, object_name):
    new = {"title": object_name}
    new["type"] = "object"
    new["properties"] = {}
    new["required"] = []
    new["description"] = object_name
    for key, value in old.items():
        new["properties"][key] = {}
        new["properties"][key]["type"] = value["type"]
        new["properties"][key]["description"] = value["description"]
        new["properties"][key]["title"] = key
        new["properties"][key]["examples"] = value["example"]
        new["properties"][key]["default"] = value["default"]
        if value["required"]:
            new["required"].append(key)
        # need to sort out string formats
    return new



In [35]:
nd = convert_to_json_schema(d, "person")

In [37]:
with open(r"c:\Users\peaco\OneDrive\Documents\GitHub\mt_metadata\examples\notebooks\person_schema.json", "w") as fid:
    json.dump(nd, fid, indent=4)

In [1]:
from mt_metadata.timeseries.person_test import Person

In [3]:
p = Person(name="steve", time="1980-01-01T00:00:00")

In [5]:
p.time = "2020-01-01 00:00:00"

In [9]:
p.model_fields["name"].description

'Persons name, should be full first and last name.'